In [91]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os

In [92]:
LATENT_DIM = 128        # 128
N_Q        = 12         # 12
N_EMB      = 64         # 64
CODEBOOK_SIZE = 512     # 512 -> 1024

In [93]:
class RVQ(nn.Module):
    """Residual Vector Quantization with multiple codebooks."""
    def __init__(self, num_quantizers=N_Q, num_embeddings=N_EMB, embedding_dim=LATENT_DIM):
        super().__init__()
        self.num_quantizers = num_quantizers
        self.embedding_dim = embedding_dim
        self.codebooks = nn.ModuleList(
            [nn.Embedding(num_embeddings, embedding_dim) for _ in range(num_quantizers)]
        )

    def forward(self, z):
        # z: (B, L, D)
        residual = z
        quantized = torch.zeros_like(z)
        all_indices = []
        for codebook in self.codebooks:
            weight = codebook.weight  # (K, D)
            # compute L2 distance
            dist = ((residual.unsqueeze(2) - weight.unsqueeze(0).unsqueeze(0))**2).sum(-1)
            indices = dist.argmin(-1)        # (B, L)
            all_indices.append(indices)
            q = F.embedding(indices, weight) # (B, L, D)
            quantized += q
            # ── γ 스케줄링: 앞단(0) 0.88 → 뒷단(7) 0.95 ──────────────────
            gamma_start, gamma_end = 0.80, 0.95
            level = len(all_indices)                      # 현재 레벨 l
            gamma = gamma_start + (gamma_end - gamma_start) * (level / (self.num_quantizers - 1))
            residual = gamma * (residual - q)
            # residual = 0.88 * (residual - q)
            # residual = residual - q
        return quantized, all_indices

In [94]:
class ConvEEGEncoder(nn.Module):
    """
    840-dim 벡터를 1×840 시퀀스로 보고 Conv1D 두 층으로 잠재표현 생성
    출력은 (B, latent_dim)
    """
    def __init__(self, input_dim=840, latent_dim=LATENT_DIM, hidden=256):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv1d(1, hidden, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv1d(hidden, latent_dim, kernel_size=3, padding=1), nn.ReLU()
        )
        self.pool = nn.AdaptiveAvgPool1d(1)   # 길이 840 → 1 로 압축

    def forward(self, x):           # x: (B, feat)
        x = x.unsqueeze(1)          # (B, 1, 840)
        z = self.conv_stack(x)      # (B, latent_dim, 840)
        z = self.pool(z).squeeze(-1)  # (B, latent_dim)
        return z

class ConvEEGDecoder(nn.Module):
    """
    latent (B,D) → (B,840)
    105→210→420→840 업샘플링
    """
    def __init__(self, output_dim=840, latent_dim=LATENT_DIM, hidden=256):
        super().__init__()
        self.start_len = 105                         # 105×8=840
        self.fc = nn.Linear(latent_dim, latent_dim * self.start_len)

        self.deconv = nn.Sequential(
            nn.ConvTranspose1d(latent_dim, hidden, kernel_size=4, stride=2, padding=1),  # 105→210
            nn.ReLU(),
            nn.ConvTranspose1d(hidden, hidden, kernel_size=4, stride=2, padding=1),      # 210→420
            nn.ReLU(),
            nn.ConvTranspose1d(hidden, 1, kernel_size=4, stride=2, padding=1)            # 420→840
        )

    def forward(self, zq):                # (B, latent)
        h = self.fc(zq).view(zq.size(0), -1, self.start_len)  # (B, latent, 105)
        x_hat = self.deconv(h)            # (B, 1, 840)
        return x_hat.squeeze(1)           # (B, 840)

In [95]:
class ConvRVQAutoEncoder(nn.Module):
    def __init__(self, feat=840, latent=LATENT_DIM, n_q=N_Q, n_emb=CODEBOOK_SIZE, hidden=256):
        super().__init__()
        self.enc = ConvEEGEncoder(feat, latent, hidden)
        self.rvq = RVQ(num_quantizers=n_q, num_embeddings=n_emb, embedding_dim=latent)
        self.dec = ConvEEGDecoder(feat, latent, hidden)

    def forward(self, x):           # x: (B, 840)
        z = self.enc(x)             # (B, latent)
        zq, indices = self.rvq(z.unsqueeze(1))  # (B, 1, D)
        zq = zq.squeeze(1)          # (B, D) — 디코더용으로 다시 압축
        x_hat = self.dec(zq)        # (B, 840)
        return x_hat, z, zq, indices

In [96]:
def rvq_loss(x, x_hat, z, z_q, beta=0.05):
    # 재구성 손실 (L_R)
    reconstruction_loss = F.mse_loss(x_hat, x)

    # 코드북 손실 (첫 번째 항 of L_VQ)
    # z는 인코더 출력 z_e(x), z_q는 양자화된 벡터 e
    # sg[z_e(x)] - e  <-->  z.detach() - z_q
    codebook_loss = F.mse_loss(z.detach(), z_q)

    # 커밋먼트 손실 (두 번째 항 of L_VQ)
    # z_e(x) - sg[e]  <-->  z - z_q.detach()
    commitment_loss = F.mse_loss(z, z_q.detach())

    # 전체 손실: L_R + L_VQ (여기서 L_VQ = codebook_loss + beta * commitment_loss)
    # 논문에서는 L_VQ의 두 항에 대한 상대적 가중치를 명시하지 않았으므로,
    # 코드북 손실의 가중치는 1로 가정합니다.
    return reconstruction_loss, codebook_loss, beta * commitment_loss

In [97]:
import glob
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class EEGVecDataset(Dataset):
    """
    EEG word-level feature 벡터 하나를 그대로 토큰으로 사용.
    """
    def __init__(self, np_array):              # np_array: (N, feat)
        mu, std = np_array.mean(0, keepdims=True), np_array.std(0, keepdims=True)+1e-8
        self.data = (np_array - mu) / std      # 정규화
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return torch.from_numpy(self.data[idx])    # shape: (feat,)

In [98]:
import random
from torch.utils.data import Sampler

class UniqueFirstSampler(Sampler):
    """
    에폭마다:
      ① unique 인덱스를 랜덤 순서로 모두 배치
      ② 남은 duplicate 인덱스를 랜덤 순서로 뒤에 붙임
    """
    def __init__(self, uniq_idx, dup_idx, shuffle=True):
        self.uniq_idx = uniq_idx
        self.dup_idx  = dup_idx
        self.shuffle  = shuffle

    def __iter__(self):
        if self.shuffle:
            random.shuffle(self.uniq_idx)
            random.shuffle(self.dup_idx)
        # unique 우선 + duplicate
        full_order = self.uniq_idx + self.dup_idx
        return iter(full_order)

    def __len__(self):
        return len(self.uniq_idx) + len(self.dup_idx)

In [99]:
# (1) 벡터 수집 --------------------------------------------------------
data_dir   = "/home/work/skku/hyo/hyo/dataset/word.parquet"
df = pd.read_parquet(data_dir)
eeg_vecs = df["eeg"].to_numpy()

arr = np.stack(eeg_vecs).astype(np.float32)
arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)

print(arr.shape, arr.dtype)  # → (N, 840) float32

unique_mask = ~df["text"].duplicated(keep="first")
unique_indices     = np.where(unique_mask)[0].tolist()
duplicate_indices  = np.where(~unique_mask)[0].tolist()

print(f"unique {len(unique_indices)}, duplicate {len(duplicate_indices)}")

# 5) Dataset & DataLoader
dataset = EEGVecDataset(arr)             # now each item is (840,) vector
# loader  = DataLoader(dataset, batch_size=256, shuffle=True)

(306211, 840) float32
unique 8855, duplicate 297356


In [100]:
class FreqAwareSampler(Sampler):
    """
    상위 top_k 빈도 단어는 keep_ratio 만큼만 사용,
    나머지는 전부 유지해 빈도를 평탄화한다.
    """
    def __init__(self, df, top_k=200, keep_ratio=0.2, shuffle=True):
        self.shuffle = shuffle
        top_words = df['text'].value_counts().head(top_k).index
        hi = df['text'].isin(top_words)
        hi_idx = df[hi].sample(frac=keep_ratio, random_state=42).index.tolist()
        lo_idx = df[~hi].index.tolist()
        self.indices = hi_idx + lo_idx

    def __len__(self):
        return len(self.indices)

    def __iter__(self):
        if self.shuffle:
            random.shuffle(self.indices)
        return iter(self.indices)

In [101]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [102]:
import faiss

def kmeans_init(model, dataloader, k=CODEBOOK_SIZE, max_samples=30000):
    """
    encoder latent 샘플 → K-means → centroids(Tensor, k×latent_dim)
    - k가 데이터 수보다 크면 k = 데이터 수 // 2 로 자동 축소
    """
    buf = []
    with torch.no_grad():
        for x in dataloader:
            buf.append(model.enc(x.to(device)))
            if len(torch.cat(buf)) >= max_samples:
                break
    latent = torch.cat(buf).cpu().numpy()         # (S, latent_dim)
    n_train = latent.shape[0]

    if n_train < k * 40:                          # faiss 권장치 미달 시 k 축소
        k_new = max(8, n_train // 40)
        print(f"⚠️  samples({n_train}) ≪ {k}×40,  k→{k_new}")
        k = k_new

    import faiss
    km = faiss.Kmeans(d=latent.shape[1], k=k, niter=20, verbose=False)
    km.train(latent)

    # 대부분의 faiss 버전은 km.centroids 속성 사용
    return torch.tensor(km.centroids).float()     # (k, latent_dim)

def usage_loss(idx_list, num_emb, lam_vec):
    losses = []
    for idx, lam in zip(idx_list, lam_vec):
        flat = idx.view(-1)
        hist = torch.bincount(flat, minlength=num_emb).float()
        p = hist / hist.sum()
        entropy = -(p[p>0] * torch.log(p[p>0])).sum()
        losses.append(-lam * entropy)
    return sum(losses)

In [103]:
model = ConvRVQAutoEncoder().to(device)

tmp_loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=0)

with torch.no_grad():
    # latent 샘플 추출
    buf = []
    for x in DataLoader(dataset, batch_size=512, shuffle=True):
        buf.append(model.enc(x.to(device)))
        if len(torch.cat(buf)) >= 50000: break
    z_full = torch.cat(buf)                     # (S,128)

    residual = z_full.clone().to(device)        # 첫 레벨 입력 (CPU 이동은 아래에서)
    for l, cb in enumerate(model.rvq.codebooks):
        lat_np = residual.cpu().numpy()
        km     = faiss.Kmeans(d=LATENT_DIM, k=CODEBOOK_SIZE, niter=15, verbose=False)
        km.train(lat_np)
        cb.weight.data.copy_(torch.tensor(km.centroids).to(device))

        # 다음 레벨 residual 계산
        weight = cb.weight                       # (K,128)
        dist   = ((residual.unsqueeze(1) - weight)**2).sum(-1)   # (S,K)
        idx    = dist.argmin(1)                  # (S,)
        q      = weight[idx]                     # (S,128)
        residual = residual - q                  # 잔차 업데이트
print("✔ all codebooks initialised (K-means per level)")

✔ all codebooks initialised (K-means per level)


In [104]:
STAGE_EPOCH = 10        # 0-1ep: level0, 2-3ep: level0-1, …
LAMBDA_USAGE = 0.05
RECON_THR  = 0.99      # 재구성 MSE가 0.50 보다 작을 때만
USAGE_THR  = -0.30     # usage_loss 가 -0.05 보다 작을 때만 (엔트로피↑)
SIGMA = 0.05              # dead-code 재초기화 범위
DEAD_THRESH = 0.12

In [105]:
import torch.optim as optim
import csv
import os

init_lr   = 1.0e-4
optimizer = optim.AdamW(model.parameters(), lr=init_lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.7,
    patience=1,
    min_lr=5e-5
)

#── 로그 파일 준비 ──
log_path = "/home/work/skku/hyo/hyo/model/rvq_best_model_word_5.log"

#── 학습 루프 ──
best_loss = float('inf')
best_model_path = "/home/work/skku/hyo/hyo/model/rvq_best_model_word_5.pt"

for epoch in range(30):
    cum_loss   = 0.0
    cum_recon  = 0.0
    cum_code   = 0.0
    cum_commit = 0.0
    cum_usage  = 0.0
    steps = 0

    # ── 3-A. 현재 활성 스테이지 계산 & freeze ──
    cur_stage = min(epoch // STAGE_EPOCH, N_Q - 1)
    for l, cb in enumerate(model.rvq.codebooks):
        cb.weight.requires_grad_(l <= cur_stage)

    # encoder warm-up(앞 2 epoch freeze)
    enc_grad = epoch >= 2
    for p in model.enc.parameters():
        p.requires_grad = enc_grad

    sampler = FreqAwareSampler(df, top_k=200, keep_ratio=0.2)
    loader = DataLoader(dataset, batch_size=256, sampler=sampler, num_workers=0)

    active_sets = [set() for _ in range(N_Q)]

    for x in loader:
        steps += 1
        x = x.to(device)

        optimizer.zero_grad()
        x_hat, z, z_q, idx_list = model(x)

        # ── active code 집계 ───────────────────────────────
        for l, idx in enumerate(idx_list):
            active_sets[l].update(idx.view(-1).tolist())

        recon_loss, codebook_loss, commit_loss = rvq_loss(x, x_hat, z, z_q)

        # lam_vec = [0.03,0.04,0.06,0.08,0.12,0.15,0.18,0.20]
        # lam_vec = [0.20,0.18,0.15,0.12,0.10,0.08,0.06,0.04]
        # lam_vec = [1.0,0.8,0.6,0.3,0.12,0.08,0.05,0.03]
        lam_vec = [1.00,0.80,0.60,0.40,0.18,0.12,0.08,0.06,0.05,0.04,0.03,0.02]
        # lam_vec = [0.25,0.20,0.15,0.12,0.10,0.08,0.06,0.05,0.04,0.03,0.02,0.01]
        # lam_vec = [0.05,0.06,0.07,0.08,0.08,0.08,0.07,0.06,0.05,0.04,0.03,0.02]
        u_loss = usage_loss(idx_list, CODEBOOK_SIZE, lam_vec)
        # u_loss = usage_loss(idx_list, CODEBOOK_SIZE, lam=LAMBDA_USAGE)

        loss = recon_loss + codebook_loss + commit_loss + u_loss
        loss.backward()
        optimizer.step()

        cum_loss   += loss.item()
        cum_recon  += recon_loss.item()
        cum_code   += codebook_loss.item()
        cum_commit += commit_loss.item()
        cum_usage  += u_loss.item()

        if steps % 30 == 0:
            avg_loss  = cum_loss   / steps
            avg_recon = cum_recon  / steps
            avg_code  = cum_code   / steps
            avg_commit= cum_commit / steps
            avg_usage = cum_usage  / steps
            with open(log_path, "a") as f:
                f.write(f"[Epoch {epoch} | Step {steps}] avg loss {avg_loss:.4f}, recon {avg_recon:.4f}, code {avg_code:.4f}, commit {avg_commit:.4f}, usage {avg_usage:.4f}\n")

    # 에폭 말 평균 계산
    avg_loss   = cum_loss   / steps
    avg_recon  = cum_recon  / steps
    avg_code   = cum_code   / steps
    avg_commit = cum_commit / steps
    avg_usage  = cum_usage  / steps
    
    # ── epoch 끝: dead-code 재초기화 ───────────────────────
    with torch.no_grad():
        for l, cb in enumerate(model.rvq.codebooks):
            ratio = len(active_sets[l]) / cb.weight.size(0)
        if ratio < DEAD_THRESH and epoch > 0:
            cb.weight.uniform_(-SIGMA, SIGMA)
            print(f"[Epoch {epoch}] L{l} reset (usage {ratio:.1%})")

    # 스케줄러 업데이트
    scheduler.step(avg_recon)
    current_lr = optimizer.param_groups[0]['lr']

    # 로그 출력
    with open(log_path, "a") as f:
        f.write(f"=== Epoch {epoch} === avg loss {avg_loss:.4f}, recon {avg_recon:.4f}, code {avg_code:.4f}, commit {avg_commit:.4f}, usage {avg_usage:.4f}, lr {current_lr:.2e}\n")
    
    # trade-off 모델 저장
    save_cond = (avg_recon < RECON_THR) and (avg_usage < USAGE_THR)
    # if save_cond and avg_loss < best_loss:
    if save_cond and avg_loss > 0:
        best_loss = avg_loss
        ckpt = {
            "encoder": model.enc.state_dict(),
            "codebooks": [cb.weight.detach().cpu() for cb in model.rvq.codebooks]
        }
        torch.save(ckpt, best_model_path)
        with open(log_path, "a") as f:
            f.write(
                f"New best model saved! "
                f"loss {best_loss:.4f}, recon {avg_recon:.4f}, usage {avg_usage:.4f}\n"
            )

In [106]:
####################################################################
# 0)  경로 & 환경
####################################################################
import torch, math, numpy as np, pandas as pd
from torch import nn
from torch.utils.data import DataLoader, Dataset
from collections import Counter
from pathlib import Path

CKPT_PATH  = Path('/home/work/skku/hyo/hyo/model/rvq_best_model_word_5.pt')
DATA_PATH  = Path('/home/work/skku/hyo/hyo/dataset/word.parquet')
BATCH_SIZE = 256
DEVICE     = 'cuda' if torch.cuda.is_available() else 'cpu'

####################################################################
# 1)  네트워크 정의 (학습 코드 그대로)
####################################################################
class RVQ(nn.Module):
    """Residual Vector Quantization with multiple codebooks."""
    def __init__(self, num_quantizers=N_Q, num_embeddings=N_EMB, embedding_dim=LATENT_DIM):
        super().__init__()
        self.num_quantizers = num_quantizers
        self.embedding_dim = embedding_dim
        self.codebooks = nn.ModuleList(
            [nn.Embedding(num_embeddings, embedding_dim) for _ in range(num_quantizers)]
        )

    def forward(self, z):
        # z: (B, L, D)
        residual = z
        quantized = torch.zeros_like(z)
        all_indices = []
        for codebook in self.codebooks:
            weight = codebook.weight  # (K, D)
            # compute L2 distance
            dist = ((residual.unsqueeze(2) - weight.unsqueeze(0).unsqueeze(0))**2).sum(-1)
            indices = dist.argmin(-1)        # (B, L)
            all_indices.append(indices)
            q = F.embedding(indices, weight) # (B, L, D)
            quantized += q
            # ── γ 스케줄링: 앞단(0) 0.88 → 뒷단(7) 0.95 ──────────────────
            gamma_start, gamma_end = 0.88, 0.95
            level = len(all_indices)                      # 현재 레벨 l
            gamma = gamma_start + (gamma_end - gamma_start) * (level / (self.num_quantizers - 1))
            residual = gamma * (residual - q)
            # residual = 0.88 * (residual - q)
            # residual = residual - q
        return quantized, all_indices

class ConvEEGEncoder(nn.Module):
    """
    840-dim 벡터를 1×840 시퀀스로 보고 Conv1D 두 층으로 잠재표현 생성
    출력은 (B, latent_dim)
    """
    def __init__(self, input_dim=840, latent_dim=LATENT_DIM, hidden=256):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv1d(1, hidden, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv1d(hidden, latent_dim, kernel_size=3, padding=1), nn.ReLU()
        )
        self.pool = nn.AdaptiveAvgPool1d(1)   # 길이 840 → 1 로 압축

    def forward(self, x):           # x: (B, feat)
        x = x.unsqueeze(1)          # (B, 1, 840)
        z = self.conv_stack(x)      # (B, latent_dim, 840)
        z = self.pool(z).squeeze(-1)  # (B, latent_dim)
        return z

####################################################################
# 2)  체크포인트 로드 → 인스턴스 복원
####################################################################
ckpt = torch.load(CKPT_PATH, map_location='cpu')

# ── encoder ──
enc_sd = ckpt['encoder']                     # state_dict (len 4)
latent_dim = enc_sd['conv_stack.2.weight'].shape[0]   # out_channels of 2nd conv
encoder = ConvEEGEncoder(input_dim=840, latent_dim=latent_dim, hidden=enc_sd['conv_stack.0.weight'].shape[0])
encoder.load_state_dict(enc_sd)
encoder = encoder.to(DEVICE).eval()

# ── RVQ ──
codebooks = ckpt['codebooks']                # list of np.arrays
n_q    = len(codebooks)
K      = codebooks[0].shape[0]
rvq    = RVQ(num_quantizers=n_q, num_embeddings=K, embedding_dim=latent_dim)
with torch.no_grad():
    for l, w in enumerate(codebooks):
        rvq.codebooks[l].weight.data.copy_(torch.tensor(w))
rvq = rvq.to(DEVICE).eval()

print(f"★ Encoder latent_dim = {latent_dim},  RVQ: {n_q}×{K} codebooks")

####################################################################
# 3)  데이터 로드 → DataLoader (raw 840-dim 벡터)
####################################################################
class EEGVecDataset(Dataset):
    """
    EEG word-level feature 벡터 하나를 그대로 토큰으로 사용.
    """
    def __init__(self, np_array):              # np_array: (N, feat)
        mu, std = np_array.mean(0, keepdims=True), np_array.std(0, keepdims=True)+1e-8
        self.data = (np_array - mu) / std      # 정규화
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return torch.from_numpy(self.data[idx])    # shape: (feat,)

df = pd.read_parquet(DATA_PATH)

eeg_vecs = df["eeg"].to_numpy()

arr = np.stack(eeg_vecs).astype(np.float32)
arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)

print(arr.shape, arr.dtype)  # → (N, 840) float32

# 5) Dataset & DataLoader
ds = EEGVecDataset(arr)             # now each item is (840,) vector
loader  = DataLoader(ds, batch_size=256, shuffle=True)

####################################################################
# 4)  평가 루프  (encoder vs. RVQ 복원)
####################################################################
mse_tot, code_hist = 0.0, Counter()
with torch.no_grad():
    for x in loader:                        # x: (B, 840)
        x = x.to(DEVICE)                    # (B, 1, 840) → (B, L=1, 840)
        z = encoder(x)                      # (B, latent)
        z = z.unsqueeze(1)
        zq, idx_list = rvq(z)               # (B, latent), (B, n_q)
        idx = torch.stack(idx_list, dim=1).squeeze(-1)
        mse_tot += torch.mean((z - zq)**2).item() * x.size(0)
        for lvl, indices in enumerate(idx.t()):
            code_hist.update([(lvl, i.item()) for i in indices])

mse  = mse_tot / len(ds)
psnr = 10 * math.log10(1.0 / mse)

####################################################################
# 5)  코드북 사용률·퍼플렉시티
####################################################################
print(f"\n==== RVQ 평가 결과 ====")
print(f"샘플 수           : {len(ds):,}")
print(f"Reconstruction MSE: {mse:.4e}")
print(f"PSNR             : {psnr:.2f} dB\n")

print("레벨별 codebook 사용률 / perplexity")
for lvl in range(n_q):
    used = [cid for (lv, cid) in code_hist if lv == lvl]
    usage = len(set(used)) / K * 100
    entropy = -sum((used.count(c)/len(used))*math.log(used.count(c)/len(used)+1e-9)
                   for c in set(used))
    perp = math.exp(entropy)
    print(f"  Level {lvl:2d}: usage {usage:6.2f}% | perplexity {perp:7.2f}")

/tmp/ipykernel_1023241/5222596.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rvq.codebooks[l].weight.data.copy_(torch.tensor(w))


★ Encoder latent_dim = 128,  RVQ: 12×512 codebooks
(306211, 840) float32

==== RVQ 평가 결과 ====
샘플 수           : 306,211
Reconstruction MSE: 1.9310e-06
PSNR             : 57.14 dB

레벨별 codebook 사용률 / perplexity
  Level  0: usage   0.20% | perplexity    1.00
  Level  1: usage   0.39% | perplexity    2.00
  Level  2: usage   1.17% | perplexity    6.00
  Level  3: usage   3.91% | perplexity   20.00
  Level  4: usage  14.26% | perplexity   73.00
  Level  5: usage  27.34% | perplexity  140.00
  Level  6: usage  36.13% | perplexity  185.00
  Level  7: usage  42.97% | perplexity  220.00
  Level  8: usage  46.68% | perplexity  239.00
  Level  9: usage  49.80% | perplexity  255.00
  Level 10: usage  56.25% | perplexity  288.00
  Level 11: usage  54.88% | perplexity  281.00
